<a href="https://colab.research.google.com/github/ukanidarshan/Projects/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('twitter_training.csv')
df.drop(columns=['2401','Borderlands'],inplace=True)
df.head()
df.columns=['label','tweet']
df.head()
df.isnull().sum()
# df.dropna(inplace=True)
# df.isnull().sum()
df.head()

,label,tweet
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
df.isnull().sum()
df = df.dropna(subset=['tweet'])
df.isnull().sum()
df.shape
df.head()

,label,tweet
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
# Converting to Lowercase
df['tweet']=df['tweet'].str.lower()
df['label']=df['label'].str.lower()
df.head()

# removing URL
import re
def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df['tweet']=df['tweet'].apply(remove_url)
df.head()

# removing punctuation
import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
df['tweet']=df['tweet'].apply(remove_punctuation)
df.head()

# removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
df['tweet']=df['tweet'].apply(remove_stopwords)
df.head()

# lemmatization the sentence
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)
df['tweet']=df['tweet'].apply(lemmatize_text)
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,label,tweet
0,positive,coming border kill
1,positive,im getting borderland kill
2,positive,im coming borderland murder
3,positive,im getting borderland 2 murder
4,positive,im getting borderland murder


In [ ]:
# using Word2Vec for feature extraction
# !pip install --upgrade numpy
!pip install --upgrade gensim


In [ ]:
!pip install numpy

In [ ]:
import gensim
from gensim.models import Word2Vec
import nltk
import pandas as pd

# Ensure 'punkt' is downloaded if 'punkt_tab' isn't available
try:
    nltk.download('punkt_tab')
except LookupError:  # If 'punkt_tab' isn't found, try 'punkt'
    nltk.download('punkt')

# Tokenizing sentences for Word2Vec
df['tokenized'] = df['tweet'].apply(nltk.word_tokenize)

# Training Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokenized'], vector_size=1000, window=5, min_count=1, workers=4)

# Save the model
word2vec_model.save('word2vec_model.model')
import numpy as np

def vectorize_sentence(sentence, model):
    words = nltk.word_tokenize(sentence)
    vectorized_words = [model.wv[word] for word in words if word in model.wv]
    if not vectorized_words:  # If no words found in Word2Vec model
        return np.zeros(model.vector_size)
    return np.mean(vectorized_words, axis=0)

# Example usage
df['vectorized'] = df['tweet'].apply(lambda x: vectorize_sentence(x, word2vec_model))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Dummy labels for the example


# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(list(df['vectorized']), df['label'], test_size=0.2, random_state=42)

# Training the model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Testing the model
y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.51


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# using tfidf for feature extraction

from sklearn.feature_extraction.text import TfidfVectorizer
# split training and testing
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)



In [ ]:
# training model with naive bayes
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X, y_train)
y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8522197445773363


In [ ]:
# use random forest for training model
# Initialize RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)

# Train the model
rf_model.fit(X, y_train)
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



Accuracy: 87.15%


In [ ]:
# training model with logistic regression

lr_model = LogisticRegression()
lr_model.fit(X, y_train)
y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8558686397729577


In [ ]:
import joblib

# Save the trained model
joblib.dump(lr_model, 'logreg_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
def predict_sentiment(text, model, vectorizer):
    # Transform the text using the same TF-IDF vectorizer
    text_tfidf = vectorizer.transform([text])

    # Predict the sentiment
    sentiment = model.predict(text_tfidf)[0]

    # Check if the model predicts an integer or string and map accordingly
    if isinstance(sentiment, int):
        sentiment_label = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        return sentiment_label[sentiment]
    elif isinstance(sentiment, str):
        return sentiment.capitalize()  # Ensure proper formatting for string outputs
    else:
        return "Unknown Sentiment"


In [ ]:
# Example test cases
test_samples = [
    "I love this product! It's amazing.",
    "I hate the customer service. It's horrible.",
    "The movie was okay, not great but not bad either."
]

# Test the model with each sample
for text in test_samples:
    sentiment = predict_sentiment(text, lr_model, tfidf_vectorizer)
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\n")


Text: I love this product! It's amazing.
Predicted Sentiment: Positive

Text: I hate the customer service. It's horrible.
Predicted Sentiment: Negative

Text: The movie was okay, not great but not bad either.
Predicted Sentiment: Neutral

